# Benchmark

In [ ]:
import os
from sys import argv

rootdir = argv[1]

#############################
#      FOR NOTEBOOK USE     #
#     SET DIRECTORY HERE    #
#                           #
# rootdir = "shmem_plot"
#                           #
#############################

print("Using root directory: {}".format(rootdir))

# Get the subdirs with the different tests
subdirs = sorted(
    [
        name
        for name in os.listdir("{}".format(rootdir))
        if os.path.isdir(os.path.join("{}".format(rootdir), name))
    ]
)
print("Available subdirs: {}".format(subdirs))

In [ ]:
import json
from sys import exit

with open("{}/settings.json".format(rootdir)) as json_file:
    settings = json.load(json_file)

print("Succesfully loaded JSON file")

## Load all files
A group that will be one line in the summarizing graph is a *node-type* + *mode* combination. This group contains the variable *rate*. See the following three groups as example:

* InfiniBand (RC): 1KHz, 10KHz, 50KHz, 100KHz
* InfiniBand (UD): 1KHz, 10KHz, 50KHz, 100KHz
* MQTT (UDP): 1KHz, 10KHz, 50KHz

## Save characteristics of tests
All important settings are contained in the name of the file. We will save them in a separate array. The structure of the name is as follows:

```bash
root_dir/benchmarks_${DATE}/${ID}_${MODE}-${VALUES IN SMP}-${RATE}-${SENT SMPS}
```

Thus, we will structure it in the settings_array as follows:

* `settings_array[*][0] = ID`
* `settings_array[*][1] = MODE`
* `settings_array[*][2] = VALUES IN SAMPLE`
* `settings_array[*][3] = RATE`
* `settings_array[*][4] = TOTAL NUMBER OF SAMPLES`

In [ ]:
import numpy as np
import re

# First, source log

# Initialize arrays
input_dataset = []
output_dataset = []
settings_array = []


for i, subdir in enumerate(subdirs):
    input_dataset.append([])
    output_dataset.append([])

    # Acquire node type from the directory
    matchObj = re.match(r"(\w*)_[A-Z]", subdir, re.M | re.I)

    # Fill value to array
    if matchObj:
        node_type = matchObj[1]

    # Acquire all tests in that subdirectory
    for walk_subdir, dirs, files in sorted(os.walk("{}/{}".format(rootdir, subdir))):
        input_dataset.append([])
        output_dataset.append([])
        settings_array.append([])

        for file in sorted(files):
            ############################
            ###### SAVE SETTINGS #######
            ############################
            temp_settings = []
            temp_settings.append(node_type)

            # Match settings, as described above
            matchObj = re.match(
                r".*?(\d*)_(\w*)-(\d*)-(\d*)-(\d*)_output.csv",
                file,
                re.M | re.I,
            )

            # Fill values to array
            if matchObj:
                for j in range(0, 5):
                    temp_settings.append(matchObj.group(j + 1))

                # Append array to big array
                settings_array[i].append(temp_settings)

            ############################
            ######### LOAD DATA ########
            ############################

            # Regex to match input files
            if re.match(r".*?_input.csv", file, re.M | re.I):
                # Load file
                input_dataset[i].append(
                    np.genfromtxt("{}/{}/{}".format(rootdir, subdir, file), delimiter=",")
                )

                print("Loaded input dataset from: {}".format(file))

            # Regex to match output files files
            elif re.match(r".*?_output.csv", file, re.M | re.I):
                output_dataset[i].append(
                    np.genfromtxt("{}/{}/{}".format(rootdir, subdir, file), delimiter=",")
                )

                print("Loaded output dataset from: {}".format(file))

    print("Settings for this subdirectory: ")
    print(settings_array[i])
    print("\n")

    # Small sanity check, are arrays of the same size?
    if len(input_dataset[i]) != len(output_dataset[i]):
        print("Error: There should be as many input files as there are output files!")
        exit();

## Get missed steps from source node
...

In [ ]:
# Number of missing samples at receive side
missed_send_arr = []
# Percentage of missed samples
perc_miss_send_arr = []

# Generate real total and number of missing samples.
# Print percentage of missed samples
for i, subdir in enumerate(subdirs):
    missed_send_arr.append([])
    perc_miss_send_arr.append([])

    for j, csv_vec in enumerate(input_dataset[i]):
        # Get number of missing samples
        missed_send_arr[i].append(int(settings_array[i][j][5]) - len(csv_vec))

        # Take percentage
        perc_miss_send_arr[i].append(
            round(missed_send_arr[i][j] / int(settings_array[i][j][5]) * 100, 2)
        )

        print(
            "Test {} ({}) is missing {} ({}%) of {} in in-file.".format(
                settings_array[i][j][0],
                settings_array[i][j][2],
                missed_send_arr[i][j],
                perc_miss_send_arr[i][j],
                settings_array[i][j][5],
            )
        )

## Get missed steps from destination node
...

In [ ]:
# Number of missing samples at receive side
missed_recv_arr = []
# Percentage of missed samples
perc_miss_recv_arr = []

# Generate real total and number of missing samples.
# Print percentage of missed samples
for i, subdir in enumerate(subdirs):
    missed_recv_arr.append([])
    perc_miss_recv_arr.append([])

    for j, csv_vec in enumerate(output_dataset[i]):
        # Get number of missing samples
        missed_recv_arr[i].append(int(settings_array[i][j][5]) - len(csv_vec))

        # Take percentage
        perc_miss_recv_arr[i].append(
            round(missed_recv_arr[i][j] / int(settings_array[i][j][5]) * 100, 2)
        )

        print(
            "Test {} ({}) is missing {} ({}%) of {} in out-file.".format(
                settings_array[i][j][0],
                settings_array[i][j][2],
                missed_recv_arr[i][j],
                perc_miss_recv_arr[i][j],
                settings_array[i][j][5],
            )
        )

## Check first and second sample from receive & destination node
...

In [ ]:
# Check first and second sample

first_second_smp_input = []
first_second_smp_output = []

for i, subdir in enumerate(subdirs):
    first_second_smp_input.append([])
    first_second_smp_output.append([])

    for j, csv_vec in enumerate(input_dataset[i]):
        first_second_smp_input[i].append([csv_vec[0][3], csv_vec[1][3]])
        print(
            "First and second sample of test {} ({}): {} and {}, respectively".format(
                settings_array[i][j][0],
                settings_array[i][j][2],
                int(first_second_smp_input[i][j][0]),
                int(first_second_smp_input[i][j][1]),
            )
        )

    for j, csv_vec in enumerate(output_dataset[i]):
        first_second_smp_output[i].append([csv_vec[0][3], csv_vec[1][3]])
        print(
            "First and second sample of test {} ({}): {} and {}, respectively".format(
                settings_array[i][j][0],
                settings_array[i][j][2],
                int(first_second_smp_output[i][j][0]),
                int(first_second_smp_output[i][j][1]),
            )
        )

    print("")

## Compare input and output data sets
...

In [ ]:
missing_seq = []

never_trans_total_arr = []
never_trans_after_arr = []

perc_never_trans_total_arr = []
perc_never_trans_after_arr = []

# Loop through input_array, since this is always bigger or equal to output array
for i, subdir in enumerate(subdirs):
    never_trans_total_arr.append([])
    never_trans_after_arr.append([])

    perc_never_trans_total_arr.append([])
    perc_never_trans_after_arr.append([])

    missing_seq.append([])

    for j, csv_vec in enumerate(input_dataset[i]):
        l = 0
        missing_seq[i].append([])
        for k, line in enumerate(csv_vec):
            try:
                if line[3] != output_dataset[i][j][l][3]:
                    missing_seq[i][j].append(line[3])
                else:
                    l += 1

            except IndexError:
                pass

        never_trans_total_arr[i].append(len(missing_seq[i][j]))

        never_trans_after_arr[i].append(
            np.sum(missing_seq[i][j] > first_second_smp_output[i][j][0])
        )

        # Take percentage
        perc_never_trans_total_arr[i].append(
            round(
                never_trans_total_arr[i][j] / int(settings_array[i][j][4]) * 100,
                2,
            )
        )
        perc_never_trans_after_arr[i].append(
            round(
                never_trans_after_arr[i][j] / int(settings_array[i][j][4]) * 100,
                2,
            )
        )

        print(
            "Test {} ({}): {} ({}%) samples were never transferred ".format(
                settings_array[i][j][0],
                settings_array[i][j][2],
                never_trans_total_arr[i][j],
                perc_never_trans_total_arr[i][j],
            )
        )
        print(
            "{} ({}%) of these after the first sample occured in out-file.".format(
                never_trans_after_arr[i][j], perc_never_trans_after_arr[i][j]
            )
        )

        print("")

## Calculate medians

In [ ]:
medians = []
upper_limit = []
lower_limit = []

for i, subdir in enumerate(subdirs):
    medians.append([])
    upper_limit.append([])
    lower_limit.append([])

    for j, csv_vec in enumerate(output_dataset[i]):
        medians[i].append(np.median(csv_vec.transpose()[2]) * 1e6)

        if settings["median_plot"]["enabled"]:
            # np.sort(recv[i][j] - enq_send[i][j])[int(np.size(recv[i][j]]) / 2)] would be the approximately the median
            # Calculate upper 10% and lower 10%
            upper_limit[i].append(
                abs(
                    medians[i][j]
                    - 1e6
                    * np.sort(csv_vec.transpose()[2])[
                        int(9 * np.size(csv_vec.transpose()[2]) / 10)
                    ]
                )
            )
            lower_limit[i].append(
                abs(
                    medians[i][j]
                    - 1e6
                    * np.sort(csv_vec.transpose()[2])[
                        int(1 * np.size(csv_vec.transpose()[2]) / 10)
                    ]
                )
            )

## Plot data
### First, define some functions

In [ ]:
# Define Fancy Box function we use
def plot_fancy_box(bottom, height, ax):
    top = bottom + height

    p = FancyBboxPatch(
        (left, bottom),
        width,
        height,
        boxstyle="round, pad=0.005",
        ec="#dbdbdb",
        fc="white",
        alpha=0.85,
        transform=ax.transAxes,
    )
    ax.add_patch(p)


# Define "find nearest" function
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

### Import all necessary libraries to plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.patches import FancyBboxPatch
from matplotlib.ticker import MultipleLocator
import pylab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib as mpl
import matplotlib.legend as mlegend

### Start with histograms if they are enabled

In [ ]:
if settings["histograms"]["enabled"]:
    for i, subdir in enumerate(subdirs):
        for j, csv_vec in enumerate(output_dataset[i]):
            # Create figure
            fig = plt.figure(
                num=None, figsize=(12, 4), dpi=90, facecolor="w", edgecolor="k"
            )

            # Add plot and set title
            ax = fig.add_subplot(111)

            # Set grid
            ax.set_axisbelow(True)
            ax.grid(True, linestyle="--")

            x_limit = 0.00005
            bins = np.arange(0, 50, 50 / 100)

            # Data in plot
            # http://www.color-hex.com/color-palette/33602
            csv_vec_t = csv_vec.transpose()

            ax.hist(
                csv_vec_t[2] * 1e6,
                label=settings["histograms"]["labels"][0],
                edgecolor="black",
                bins=bins,
                color="#00549f",
            )
            ax.axvline(
                medians[i][j],
                color="red",
                linestyle="-",
                linewidth=1,
                alpha=0.85,
            )

            # Set axis and calculate values above limit
            plt.xlim([0, x_limit])

            ###################################
            # SET TICKS #######################
            ###################################
            ticks = np.arange(0, x_limit * 1e6 + 1, 2)

            nearest, nearest_idx = find_nearest(ticks, medians[i][j])

            ticks = np.append(ticks, medians[i][j])

            # Explicitly set labels
            labels = []

            for value in ticks:
                if value == nearest and np.abs(nearest - medians[i][j]) < 200:
                    labels.append("")
                elif value == (medians[i][j]):
                    labels.append(value)
                else:
                    labels.append(str(int(value)))

            plt.yticks(fontsize=10, family="monospace")
            plt.xticks(
                ticks,
                labels,
                fontsize=10,
                family="monospace",
                rotation=30,
                horizontalalignment="right",
                rotation_mode="anchor",
            )

            for value in ax.get_xticklabels():
                try:
                    if int(float(value.get_text())) == int(medians[i][j]):
                        value.set_color("red")
                except ValueError:
                    # We got some empty values. Ignore them
                    pass

            minorLocator = MultipleLocator(1)
            ax.xaxis.set_minor_locator(minorLocator)

            ###################################
            # CONFIGURE AXIS ##################
            ###################################
            # Set labels
            ax.set_xlabel(
                settings["histograms"]["axis_labels"]["x"],
                fontsize=10,
                family="monospace",
                labelpad=4,
            )
            ax.set_ylabel(
                settings["histograms"]["axis_labels"]["y"],
                fontsize=10,
                family="monospace",
                labelpad=6,
            )

            # Set scale
            ax.set_yscale("log")

            ###################################
            # CREATE TEXTBOXES ################
            ###################################
            off_bigger_50us = round(
                (np.size(csv_vec_t[2][csv_vec_t[2] > x_limit]) / np.size(csv_vec_t[2]))
                * 100,
                2,
            )

            offset_text = "$\mathtt{{t_{{lat}}>50µs: }}${0: >5.2f}% ($\mathtt{{\\max\\,t_{{lat}}}}$: {1:>7.2f}µs)".format(
                off_bigger_50us, round(np.max(csv_vec_t[2]) * 1e6, 2)
            )

            # Create text for missed steps
            missed_text = " in: {0:6d} ({1:5.2f}%)\n".format(
                missed_send_arr[i][j], perc_miss_send_arr[i][j]
            )
            missed_text += "out: {0:6d} ({1:5.2f}%)".format(
                missed_recv_arr[i][j], perc_miss_recv_arr[i][j]
            )

            # Create text for missed steps
            never_transferred_text = "total: {0:5d} ({1:5.2f}%)\n".format(
                never_trans_total_arr[i][j], perc_never_trans_total_arr[i][j]
            )
            never_transferred_text += "while connected: {0:5d} ({1:5.2f}%)".format(
                never_trans_after_arr[i][j],
                perc_never_trans_after_arr[i][j],
            )

            # Set font properties for headers and text
            font_header = FontProperties()
            font_header.set_family("monospace")
            font_header.set_weight("bold")
            font_header.set_size(9.5)

            font_text = FontProperties()
            font_text.set_size(9.5)
            font_text.set_family("monospace")

            # Set box constraints for wrapper and plot wrapper
            left, width = 0.673, 0.33
            right = left + width

            plot_fancy_box(bottom=0.46, height=0.65, ax=ax)

            # Set box constraints for text boxes
            left, width = 0.685, 0.30
            right = left + width

            # Offset boxes
            plot_fancy_box(bottom=0.9085, height=0.085, ax=ax)

            ax.text(
                right,
                0.975,
                offset_text,
                verticalalignment="top",
                horizontalalignment="right",
                transform=ax.transAxes,
                color="black",
                fontproperties=font_text,
            )

            # Missed steps
            plot_fancy_box(bottom=0.695, height=0.18, ax=ax)

            ax.text(
                right,
                0.868,
                "missing samples:",
                verticalalignment="top",
                horizontalalignment="right",
                transform=ax.transAxes,
                color="black",
                fontproperties=font_header,
            )
            ax.text(
                right,
                0.804,
                missed_text,
                verticalalignment="top",
                horizontalalignment="right",
                transform=ax.transAxes,
                color="black",
                fontproperties=font_text,
            )

            # Never transferred
            plot_fancy_box(bottom=0.487, height=0.175, ax=ax)

            ax.text(
                right,
                0.657,
                "samples not transmitted:",
                verticalalignment="top",
                horizontalalignment="right",
                transform=ax.transAxes,
                color="black",
                fontproperties=font_header,
            )
            ax.text(
                right,
                0.593,
                never_transferred_text,
                verticalalignment="top",
                horizontalalignment="right",
                transform=ax.transAxes,
                color="black",
                fontproperties=font_text,
            )

            ###################################
            # SAVE PLOT #######################
            ###################################
            plt.minorticks_on()
            plt.tight_layout()

            fig.savefig(
                "{}/{}_{}_{}i_{}j.pdf".format(
                    rootdir,
                    settings_array[i][j][0],
                    settings_array[i][j][2],
                    i,
                    j,
                ),
                format="pdf",
            )

    ###################################
    # CREATE HISTOGRAM LEGEND #########
    ###################################
    # create a second figure for the legend
    figLegend = pylab.figure(figsize=settings["histograms"]["dimensions"]["legend"])

    # produce a legend for the objects in the other figure
    pylab.figlegend(
        *ax.get_legend_handles_labels(),
        loc="upper left",
        prop={"family": "monospace", "size": "8"},
        ncol=settings["histograms"]["legend_columns"]
    )

    figLegend.savefig("{}/legend_histogram.pdf".format(rootdir), format="pdf")

In [ ]:
if settings["median_plot"]["enabled"]:
    # Create figure and axis
    fig_median = plt.figure(
        num=None, figsize=(12, 4), dpi=90, facecolor="w", edgecolor="k"
    )
    ax_median = fig_median.add_subplot(111)

    for i, subdir in enumerate(subdirs):
        ###################################
        # CREATE MEDIAN PLOT ##############
        ###################################
        x_data = np.array([])
        for k in range(0, len(medians[i])):
            x_data = np.append(x_data, k)

        ax_median.errorbar(
            x_data,
            medians[i],
            yerr=[lower_limit[i], upper_limit[i]],
            capsize=3.7,
            elinewidth=1,
            markeredgewidth=1,
            marker="v",
            zorder=2 + i,
            color=settings["median_plot"]["colors"][i],
            label=settings["median_plot"]["labels"][i],
        )

        ###################################
        # PRINT MISSED STEPS ##############
        ###################################
        if settings["median_plot"]["print_missed_steps"]:
            for l, median in enumerate(medians[i]):
                p = FancyBboxPatch(
                    (x_data[l] + 0.07, median + 0.08),
                    0.345,
                    0.26,
                    boxstyle="round, pad=0.005",
                    ec="#dbdbdb",
                    fc="white",
                    alpha=0.85,
                )
                ax_median.add_patch(p)

                ax_median.text(
                    x_data[l] + 0.1,
                    median + 0.15,
                    "{: >4.2f}%".format(perc_miss_recv_arr[i][l]),
                )

            # Create bbox patch for legend
            # p = FancyBboxPatch((0, 0), 5, 1, boxstyle="round, pad=0.5", ec="#dbdbdb", fc="white", alpha=0.85)

            handles = []
            handles.append(p)
            text = "% of samples missed by signal generator"
            leg2 = mlegend.Legend(
                ax_median,
                handles,
                labels=[text],
                loc="upper left",
                ncol=1,
                prop={"family": "monospace", "size": "8"},
            )

    ###################################
    # SET AXIS OF MEDIAN PLOT #########
    ###################################
    ax_median.set_xticks(np.arange(0, len(settings["median_plot"]["ticks"]["x"]), 1))
    ax_median.set_xticklabels(settings["median_plot"]["ticks"]["x"])

    if settings["median_plot"]["log_scale"]:
        ax_median.set_yscale("log")
    else:
        ax_median.set_ylim(
            [
                settings["median_plot"]["ticks"]["y"][0],
                settings["median_plot"]["ticks"]["y"][-1],
            ]
        )
        ax_median.set_yticks(settings["median_plot"]["ticks"]["y"])

    ax_median.set_xlabel(
        settings["median_plot"]["axis_labels"]["x"],
        fontsize=11,
        family="monospace",
        labelpad=6,
    )
    ax_median.set_ylabel(
        settings["median_plot"]["axis_labels"]["y"],
        fontsize=11,
        family="monospace",
        labelpad=6,
    )
    ax_median.set_axisbelow(True)
    ax_median.grid(True, linestyle="--")

    ax_median.yaxis.grid(True, linestyle="-", which="major", color="black", alpha=0.8)
    ax_median.yaxis.grid(
        True, linestyle="--", which="minor", color="lightgrey", alpha=0.3
    )

    ###################################
    # EXPORT MEDIANS AND CREATE #######
    # LEGEND OF MEDIAN TABLE ##########
    ###################################
    plt.tight_layout()
    fig_median.savefig(
        "{}/median_graph.pdf".format(rootdir),
        dpi=600,
        format="pdf",
        bbox_inches="tight",
    )

    # create a second figure for the legend
    figLegend = pylab.figure(figsize=settings["median_plot"]["dimensions"]["legend"])

    leg_temp = pylab.figlegend(
        *ax_median.get_legend_handles_labels(),
        loc="upper left",
        labelspacing=1.2,
        prop={"family": "monospace", "size": "8"},
        ncol=settings["median_plot"]["legend_columns"]
    )

    if settings["median_plot"]["print_missed_steps"]:
        leg_temp._legend_box._children.append(leg2._legend_box._children[1])
        leg_temp._legend_box.align = "left"

    figLegend.savefig("{}/legend_median_plot.pdf".format(rootdir), format="pdf")

## Create 3D-Plot if enabled

In [ ]:
if settings["3d_plot"]["enabled"]:
    for i, subdir in enumerate(subdirs):
        fig_3d = plt.figure(
            num=None, figsize=(16, 7), dpi=90, facecolor="w", edgecolor="k"
        )
        ax_3d = fig_3d.gca(projection="3d")

        # Make data.
        X = np.array([])
        for k in range(0, len(settings["3d_plot"]["ticks"]["x"])):
            X = np.append(X, k)

        Y = np.array([])
        for k in range(0, len(settings["3d_plot"]["ticks"]["y"])):
            Y = np.append(Y, k)

        X, Y = np.meshgrid(X, Y)

        Z = np.array([])
        for k in range(0, len(settings["3d_plot"]["ticks"]["y"])):
            for l in range(0, len(settings["3d_plot"]["ticks"]["x"])):
                Z = np.append(
                    Z,
                    medians[i][k * len(settings["3d_plot"]["ticks"]["x"]) + l],
                )

        ###################################
        # PRINT MISSED STEPS ##############
        ###################################

        props = dict(boxstyle="round", facecolor="white", alpha=0.8)

        # if more than 5% of the samples were missed, print it to figure
        for k in range(0, len(input_dataset[i])):
            if perc_miss_send_arr[i][k] > 5:
                x = k % (len(settings["3d_plot"]["ticks"]["x"]))
                y = np.floor(k / (len(settings["3d_plot"]["ticks"]["y"])))
                z = Z[k]

                x_delta = 0.65
                y_delta = 0.65
                z_delta = 0.5 * (Z[k] - Z[k - len(settings["3d_plot"]["ticks"]["y"])])
                ax_3d.text(
                    x - x_delta,
                    y - y_delta,
                    z - z_delta,
                    "{: >4.2f}%".format(perc_miss_send_arr[i][k]),
                    fontsize=11,
                    family="monospace",
                    color="red",
                    bbox=props,
                )

        Z = np.split(Z, len(settings["3d_plot"]["ticks"]["y"]))

        # Plot the surface.
        surf = ax_3d.plot_surface(
            X, Y, Z, cmap=cm.Blues, linewidth=135, antialiased=False, shade=True
        )
        ax_3d.plot_wireframe(X, Y, Z, 10, lw=1, colors="k", linestyles="solid")

        # Customize the z axis.
        ax_3d.set_zlim(0, np.max(np.ceil(Z)))
        ax_3d.zaxis.set_major_locator(LinearLocator(10))

        ax_3d.set_xlabel(
            settings["3d_plot"]["axis_labels"]["x"],
            fontsize=11,
            family="monospace",
            labelpad=14,
        )
        ax_3d.set_ylabel(
            settings["3d_plot"]["axis_labels"]["y"],
            fontsize=11,
            family="monospace",
            labelpad=8,
        )
        ax_3d.set_zlabel(
            settings["3d_plot"]["axis_labels"]["z"],
            fontsize=11,
            family="monospace",
            labelpad=8,
        )

        ax_3d.set_xticks(np.arange(0, len(settings["3d_plot"]["ticks"]["x"]), 1))
        ax_3d.set_xticklabels(settings["3d_plot"]["ticks"]["x"])

        ax_3d.set_yticklabels(settings["3d_plot"]["ticks"]["y"])
        ax_3d.set_zticks(np.arange(0, len(settings["3d_plot"]["ticks"]["z"]), 1))

        x = np.argmin(Z) % (len(settings["3d_plot"]["ticks"]["x"]))
        y = np.floor(np.argmin(Z) / (len(settings["3d_plot"]["ticks"]["y"])))
        z = np.min(Z)

        ax_3d.plot(
            [x, x],
            [y, y],
            z,
            marker="v",
            color="green",
            markersize=15,
            label="Minimum: " + str(z) + " µs",
        )

        x = np.argmax(Z) % (len(settings["3d_plot"]["ticks"]["x"]))
        y = np.floor(np.argmax(Z) / (len(settings["3d_plot"]["ticks"]["y"])))
        z = np.max(Z)

        ax_3d.plot(
            [x, x],
            [y, y],
            z,
            marker="^",
            color="red",
            markersize=15,
            label="Maximum: " + str(z) + " µs",
        )

        norm = mpl.colors.Normalize(vmin=np.min(Z), vmax=np.max(Z))
        cb = fig_3d.colorbar(surf, shrink=0.8, aspect=10, fraction=0.1, norm=norm)
        cb.set_label(
            settings["3d_plot"]["axis_labels"]["z"],
            fontsize=11,
            family="monospace",
            labelpad=8,
        )
        plt.tight_layout()
        plt.show()

        fig_3d.savefig(
            "{}/median_3d_graph_{}.pdf".format(rootdir, settings_array[i][0][2]),
            dpi=600,
            format="pdf",
        )

        ###################################
        # CREATE LEGEND ###################
        ###################################
        # create a second figure for the legend
        figLegend = pylab.figure(figsize=settings["3d_plot"]["dimensions"]["legend"])

        # The markers are too big, so lets create smaller markers
        ax_custom = figLegend.add_subplot(111)
        ax_custom.plot(
            0,
            0,
            marker="v",
            color="green",
            label="$\\min\\,\\tilde{t}_{lat}$: " + str(np.min(Z)) + " µs",
            markersize=8,
            linestyle="None",
        )
        ax_custom.plot(
            0,
            0,
            marker="^",
            color="red",
            label="$\\max\\,\\tilde{t}_{lat}$: " + str(np.max(Z)) + " µs",
            markersize=8,
            linestyle="None",
        )
        ax_custom.set_visible(False)

        # Create bbox patch for legend
        p = FancyBboxPatch(
            (0, 0),
            5,
            1,
            boxstyle="round, pad=0.5",
            ec="#dbdbdb",
            fc="white",
            alpha=0.85,
        )

        handles = []
        handles.append(p)
        text = "% of samples missed by signal generator"
        leg2 = mlegend.Legend(
            ax_custom,
            handles,
            labels=[text],
            loc="upper left",
            ncol=1,
            prop={"family": "monospace", "size": "8"},
        )

        # Extract handles from pseudo plot
        handles, labels = ax_custom.get_legend_handles_labels()

        leg_temp = pylab.figlegend(
            handles,
            labels,
            loc="upper left",
            labelspacing=1.2,
            prop={"family": "monospace", "size": "8"},
            ncol=settings["3d_plot"]["legend_columns"],
        )

        # Concat handles
        leg_temp._legend_box._children.append(leg2._legend_box._children[1])
        leg_temp._legend_box.align = "left"

        # Save figure
        figLegend.savefig(
            "{}/legend_median_3d_plot_{}.pdf".format(rootdir, settings_array[i][0][2]),
            format="pdf",
        )